In [2]:
from datetime import datetime
import pandas as pd
import numpy as np

from csv import DictReader
from tqdm import tqdm

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
./FM --bias true --linear true --task_type сlassification -t avazu-app.tr -e avazu-app.val  --pairwise 0 --learning_rate 0.002 --learning_method SGD --regularization_const 0.01 -g memory --hash_size 30000 -g inplace -i 10

In [3]:
!head -n 3 SourceData/avazu_train.csv

## Честные фичи для VW

In [ ]:
from datetime import datetime
from csv import DictReader

def csv_to_vw(loc_csv, loc_output):
    start = datetime.now()
  
    with open(loc_output,"w") as outfile:
        for row in tqdm( DictReader(open(loc_csv)) ):
            numerical_features = ""
            categorical_features = ""
            for k,v in row.items():
                if k not in ["id","click"]:
                    if len(str(v)) > 0:
                        categorical_features += " %s" % v
            if row['click'] == "1":
                label = 1
            else:
                label = -1 #we set negative label to -1
            outfile.write( "%s '%s |i%s |c%s\n" % (label,row['id'],numerical_features,categorical_features) )

    print("\n %s Task execution time:\n\t%s"%(e, str(datetime.now() - start)))

csv_to_vw("SourceData/avazu_train.csv", "SourceData/avazu_source_train.vw")
!head -n 3 SourceData/avazu_source_train.vw

35891414it [30:40, 19496.28it/s]

## Хэш фичи для FM

In [3]:
def csv_to_fm(loc_csv, loc_output, hash_size):
    start = datetime.now()
    with open(loc_output,"w") as outfile:
        for row in tqdm( DictReader(open(loc_csv)) ):
            features = ""
            for k,v in row.items():
                if k not in ["id", "click"]:
                    if len(str(v)) > 0:
                        features += "%s:1 " % (hash('{0}_{1}'.format(k, v)) % hash_size)
            if row['click'] == "1":
                label = 1
            else:
                label = 0 #we set negative label to 0
            outfile.write("{0} {1}\n".format(label, features))

    print("\n %s Task execution time:\n\t%s"%(e, str(datetime.now() - start)))

csv_to_fm("SourceData/avazu_train.csv", "SourceData/avazu_train.fm", 2 ** 31)

0	0:00:02.184052
100000	0:00:09.015549
200000	0:00:15.873615
300000	0:00:22.338868
400000	0:00:28.831681
500000	0:00:35.813975
600000	0:00:42.255359
700000	0:00:48.632769
800000	0:00:55.074886
900000	0:01:01.583021
1000000	0:01:08.503991
1100000	0:01:13.519886
1200000	0:01:19.129297
1300000	0:01:26.650159
1400000	0:01:34.872204
1500000	0:01:42.353815
1600000	0:01:49.297583
1700000	0:01:56.620551
1800000	0:02:03.694036
1900000	0:02:14.267014
2000000	0:02:21.494238
2100000	0:02:29.589253
2200000	0:02:35.884206
2300000	0:02:43.537245
2400000	0:02:51.054308
2500000	0:02:58.974475
2600000	0:03:09.406961
2700000	0:03:20.561968
2800000	0:03:32.921124
2900000	0:03:41.765301
3000000	0:03:49.875460
3100000	0:03:57.128115
3200000	0:04:03.218537
3300000	0:04:10.744902
3400000	0:04:17.855835
3500000	0:04:25.024707
3600000	0:04:32.523004
3700000	0:04:39.636071
3800000	0:04:47.116087
3900000	0:04:53.390857
4000000	0:04:59.531885
4100000	0:05:06.497368
4200000	0:05:13.545783
4300000	0:05:20.230754
440

34700000	0:40:10.414146
34800000	0:40:17.949223
34900000	0:40:25.421186
35000000	0:40:32.450883
35100000	0:40:38.962080
35200000	0:40:46.510941
35300000	0:40:54.345626
35400000	0:41:02.037923
35500000	0:41:09.328428
35600000	0:41:15.369295
35700000	0:41:21.249163
35800000	0:41:26.955209
35900000	0:41:33.813484
36000000	0:41:40.329486
36100000	0:41:46.194932
36200000	0:41:52.650158
36300000	0:41:59.805248
36400000	0:42:07.589330
36500000	0:42:15.659139
36600000	0:42:23.772245
36700000	0:42:31.953014
36800000	0:42:39.172801
36900000	0:42:47.166049
37000000	0:42:55.745236
37100000	0:43:04.744695
37200000	0:43:11.488389
37300000	0:43:18.309948
37400000	0:43:25.313808
37500000	0:43:32.823966
37600000	0:43:39.349720
37700000	0:43:45.864669
37800000	0:43:52.431074
37900000	0:43:58.790578
38000000	0:44:05.672233
38100000	0:44:12.439958
38200000	0:44:18.879909
38300000	0:44:25.548452
38400000	0:44:32.494773
38500000	0:44:39.093277
38600000	0:44:45.690339
38700000	0:44:52.765041
38800000	0:44:59

In [6]:
def fm2vw(infile, outfile):    
    input_file = open(infile,  'r')
    out_file   = open(outfile, 'w')
    for line in tqdm(input_file):
        target = str(int(line[0]) * 2 - 1) 
        out_file.write(target + ' |' + line[1:])
        
fm2vw('SourceData/avazu_train.fm', 'SourceData/avazu_train.vw')

40428967it [02:50, 237091.24it/s]


In [7]:
!head -n 3 SourceData/avazu_train.vw

-1 | 156565304:1 1513862964:1 1716396866:1 1301993812:1 1563783598:1 360217669:1 315341543:1 2086032928:1 1558635273:1 896241932:1 1385758995:1 294760084:1 1984940700:1 1304100402:1 801839287:1 95123933:1 1526537554:1 840809851:1 510857813:1 220749133:1 469727479:1 637391510:1 
-1 | 156565304:1 1513862964:1 1716396866:1 1301993812:1 1563783598:1 360217669:1 315341543:1 2086032928:1 1558635273:1 896241932:1 1803468718:1 157830478:1 1984940700:1 110206437:1 1065627487:1 95123933:1 1526537554:1 840809851:1 510857813:1 220749133:1 631055937:1 637391510:1 
-1 | 156565304:1 1513862964:1 1716396866:1 1301993812:1 1563783598:1 360217669:1 315341543:1 2086032928:1 1558635273:1 896241932:1 371073318:1 77647503:1 1984940700:1 110206437:1 1065627487:1 95123933:1 1526537554:1 840809851:1 510857813:1 220749133:1 631055937:1 637391510:1 


In [6]:
!vw SourceData/avazu_train.vw -f VwHelpFiles/model3 --loss_function logistic --sgd --learning_rate 0.001 -b 18 --passes 1 --cache_file VwHelpFiles/cache3

final_regressor = VwHelpFiles/model3
Num weight bits = 18
learning rate = 0.001
initial_t = 1
power_t = 0.5
creating cache_file = VwHelpFiles/cache3
Reading datafile = SourceData/avazu_train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.693147 0.693147            1            1.0  -1.0000   0.0000       23
0.691559 0.689970            2            2.0  -1.0000  -0.0064       23
0.688636 0.685714            4            4.0  -1.0000  -0.0174       23
0.686493 0.684350            8            8.0  -1.0000  -0.0196       23
0.685060 0.683627           16           16.0  -1.0000  -0.0383       23
0.681721 0.678382           32           32.0  -1.0000  -0.0475       23
0.681872 0.682024           64           64.0  -1.0000  -0.0691       23
0.676512 0.671151          128          128.0  -1.0000  -0.0659       23
0.667954 0.659396          256          256.0   1.0000  -0.0

In [ ]:
def kfold(infile, out_file_name, nfolds=5):
    